<h3>03-01. 축 기준 (apply)</h3>

DataFrame.apply(func, axis=0, raw=False, result_type=None, args=(), **kwargs)

function : 각 행이나 열에 적용할 함수 입니다.</br>
axis : {0 : Index / 1 : columns} 함수를 적용할 축 입니다.</br>
row : {True : ndarray / False : Series} 함수에 전달할 축의 형식입니다.</br>
True면 ndarray형태로 전달하고 False면 Series형태로 전달합니다. 기본적으로 Series입니다.</br>
result_type : {expand / reduce / broadcast} 반환값의 형태를 결정합니다. expand이면 배열 형태를
기준으로 열을 확장합니다.(기본 인덱스로), reduce인 경우는 그대로 Serise형태로 반환합니다.</br>
broadcase인 경우 기존 열 형식대로 확장하여 반환합니다.(열의 수가 같아야합니다.)</br>

In [6]:
import pandas as pd
import numpy as np

col = ['col1','col2','col3']
row = ['row1','row2','row3']
data = [[1,2,3],[4,5,6],[7,8,9]]
df = pd.DataFrame(data=data,index=row,columns=col)
print(df)

      col1  col2  col3
row1     1     2     3
row2     4     5     6
row3     7     8     9


In [7]:
print(df.apply(np.sqrt))

          col1      col2      col3
row1  1.000000  1.414214  1.732051
row2  2.000000  2.236068  2.449490
row3  2.645751  2.828427  3.000000


In [9]:
print(df.apply(np.sum))

col1    12
col2    15
col3    18
dtype: int64


In [11]:
print(df.apply(np.prod,axis=0))

col1     28
col2     80
col3    162
dtype: int64


In [12]:
print(df.apply(np.prod,axis=1))

row1      6
row2    120
row3    504
dtype: int64


In [15]:
print(df.apply(lambda x : [1,2,3]))

      col1  col2  col3
row1     1     1     1
row2     2     2     2
row3     3     3     3


In [16]:
#result_type = 'expand'인 경우
#func를 기준으로 확장하여 columns를 지정하게 되는것을 확인할 수 있습니다.
print(df.apply(lambda x : [1,2,3], axis=1,result_type='expand'))

      0  1  2
row1  1  2  3
row2  1  2  3
row3  1  2  3


In [18]:
#result_type = 'reduce'인 경우
#func를 기준으로 축소하여 columns없이 Series 객체로 반환하는것을 확인할 수 있습니다.
print(df.apply(lambda x : [1,2,3], axis=1,result_type='reduce'))

row1    [1, 2, 3]
row2    [1, 2, 3]
row3    [1, 2, 3]
dtype: object


In [20]:
#result_type = 'broadcast'인 경우
#func를 기준으로 확장하되, columns는 기존 DataFrame의 것을 사용하는것을 확인할 수 있습니다.
print(df.apply(lambda x : [1,2,3], axis=1,result_type='broadcast'))

      col1  col2  col3
row1     1     2     3
row2     1     2     3
row3     1     2     3


<h3>03-02. 요소별 (applymap)</h3>

DataFrame.applymap(func, na_action=None, **kwargs)

func : 단일 값을 반환하는 함수 입니다.</br>
na_action : {None / 'ignore} NaN의 무시 여부입니다. 'ignore'이면 NaN을 함수로 전달하지 않습니다.</br>

In [23]:
col = ['col1','col2','col3']
row = ['row1','row2','row3']
data = [[1,2,3],[4,5,6],[7,pd.NA,9]]
df = pd.DataFrame(data=data,index=row,columns=col)
print(df)

      col1  col2  col3
row1     1     2     3
row2     4     5     6
row3     7  <NA>     9


In [26]:
print(df.applymap(lambda x : x**2,na_action='ignore'))

      col1  col2  col3
row1     1     4     9
row2    16    25    36
row3    49  <NA>    81


<h3>03-03. 함수내 함수 연속적용 (pipe)</h3>

DataFrame.pipe(func, args, kwargs)

func : 함수입니다.</br>
arg : 함수의 인수입니다.</br>
kwargs : dict 형태의 함수의 인수입니다.</br>

만약 함수 3개가 아래와 같이 있다고 해봅니다.</br>
f1(data, arg1), f2(data, arg1, arg2, f3(data, arg3)
f1 > f2 > f3 순서로 포함되게 함수를 사용한다고 하면 아래와 같이 함수를 사용해야 합니다.</br>
df=f1( f2( f3( data,arg3='c' ),arg2='b1',arg3='b2' ),arg1='a' )
이는 어떤 arg가 어떤함수인지 직관적으로 볼 수 없습니다. 이때, pipe함수를 사용할 수 있습니다.</br>
df=data.pipe(f3, arg3='c').pipe(f2, arg2='b1', arg3='b2').pipe(f3, arg3='c')</br>


In [28]:
org_data = pd.DataFrame({'info':['삼성전자/3/70000','SK하이닉스/2/100000']})
print(org_data)

              info
0     삼성전자/3/70000
1  SK하이닉스/2/100000


In [61]:
def code_name(data):
    result = pd.DataFrame(columns = ['name','count','price'])
    df = pd.DataFrame(list(data['info'].str.split('/')))
    result['name'] = df[0]
    result['count'] = df[1]
    result['price'] = df[2]
    result = result.astype({'count':int,'price':int})
    return result

print(code_name(org_data))

     name  count   price
0    삼성전자      3   70000
1  SK하이닉스      2  100000


In [63]:
def value_cal(data,unit=''):
    result = pd.DataFrame(columns=['name','value']) 
    result['name'] =data['name'] # 이름은 기존거를 가져옴
    result['value']=data['count']*data['price'] # value는 count * price를 입력함
    result = result.astype({'value':str}) # value를 str로 변경(단위를 붙이기 위함)
    result['value']=result['value']+unit # 단위를 붙임
    return(result)

input=code_name(org_data)
print(value_cal(input,'원'))

     name    value
0    삼성전자  210000원
1  SK하이닉스  200000원


In [64]:
print(org_data.pipe(code_name).pipe(value_cal,'원'))

     name    value
0    삼성전자  210000원
1  SK하이닉스  200000원


<h3>03-04. 함수연속적용_축별 (aggregate, agg)</h3>

DataFrame.aggregate(func=None, axis=0, args, kwargs)
DataFrame.agg(func=None, axis=0, args, kwargs)

func : 함수입니다.</br>
axis :{0 : index(row) / 1 : columns} 축입니다 0은 행, 1은 열 입니다. arg : 함수의 인수 입니다..</br>
kwargs : dict 형태의 함수의 인수입니다.</br>

In [66]:
df = pd.DataFrame([[1,4,7],[2,5,8],[3,6,9]])
print(df)

   0  1  2
0  1  4  7
1  2  5  8
2  3  6  9


In [67]:
#입력되는 함수의 형태에 따라
#입력함수로는 먼저 np.함수 형태나 그냥 문자열 형태로의 입력이 가능합니다.
#np함수의 경우
ex1 = df.agg(np.prod)
print(ex1)

0      6
1    120
2    504
dtype: int64


In [71]:
#문자열일 경우
ex2 = df.agg('prod')
print(ex2)

0      6
1    120
2    504
dtype: int64


In [72]:
ex3 = df.agg([lambda x : min(x) * max(x)])
print(ex3)

          0   1   2
<lambda>  3  24  63


In [73]:
def func_sub(input):
    return max(input)-min(input)
ex4 = df.agg([func_sub,'sum'])
print(ex4)

          0   1   2
func_sub  2   2   2
sum       6  15  24


In [74]:
def func_sub(input):
    return max(input)-min(input)
func_sub.__name__='내함수'
ex5 = df.agg([func_sub,'sum'])
print(ex5)

     0   1   2
내함수  2   2   2
sum  6  15  24


In [75]:
ex6 = df.agg(['min','max','sum','prod'])
print(ex6)

      0    1    2
min   1    4    7
max   3    6    9
sum   6   15   24
prod  6  120  504


In [76]:
ex7 = df.agg({2:'sum',0:'max',1:'min'})
print(ex7)

2    24
0     3
1     4
dtype: int64


In [78]:
ex8 = df.agg({0:['sum','prod'],1:['max','min'],2:'mean'})
print(ex8)

        0    1    2
sum   6.0  NaN  NaN
prod  6.0  NaN  NaN
max   NaN  6.0  NaN
min   NaN  4.0  NaN
mean  NaN  NaN  8.0


In [80]:
#dict함수내에 다중함수를 적용할 수도 있습니다. 이 경우 해당되지 않는 index는 NaN을 출력합니다.
ex8 = df.agg({0:['sum','prod'],1:['max','min'],2:'mean'})
print(ex8)

        0    1    2
sum   6.0  NaN  NaN
prod  6.0  NaN  NaN
max   NaN  6.0  NaN
min   NaN  4.0  NaN
mean  NaN  NaN  8.0


In [81]:
#axis인수를 변경할 경우
#axis의경우 0이면 index, 1이면 columns 기준으로 연산을 진행합니다.
ex2 = df.agg('prod', axis=0)
print(ex2)

0      6
1    120
2    504
dtype: int64


In [82]:
ex3 = df.agg('prod', axis=1)
print(ex3)

0     28
1     80
2    162
dtype: int64


<h3>03-05. 함수연속적용_요소별 (transform)</h3>

DataFrame.transform(func, axis=0, args, kwargs)

transform메서드는 agg와 비슷하게 함수를 적용하는 메서드이지만,</br>
단일 요소별로 함수를 동시에 적용할 수 있다는 장점이 있습니다. 마치 apply와 applymap의 차이와 비슷합니다.</br>

func : 함수입니다.</br>
axis :{0 : index(row) / 1 : columns} 축입니다 0은 행, 1은 열 입니다.</br>
arg : 함수의 인수 입니다.</br>
kwargs : dict 형태의 함수의 인수입니다.</br>

In [84]:
col = ['col1','col2','col3']
row = ['row1','row2','row3']
df = pd.DataFrame(data=[[10,40,70],[20,50,80],[30,60,90]],index=row,columns=col)
print(df)

      col1  col2  col3
row1    10    40    70
row2    20    50    80
row3    30    60    90


In [87]:
#입력되는 함수의 형태에 따라 입력함수로는 먼저 np.함수 형태나 그냥 문자열 형태로의 입력이 가능합니다.
#np함수의 경우
ex1 = df.transform(np.sqrt)
print(ex1)

          col1      col2      col3
row1  3.162278  6.324555  8.366600
row2  4.472136  7.071068  8.944272
row3  5.477226  7.745967  9.486833


In [88]:
ex2 = df.transform('sqrt')
print(ex2)

          col1      col2      col3
row1  3.162278  6.324555  8.366600
row2  4.472136  7.071068  8.944272
row3  5.477226  7.745967  9.486833


In [89]:
ex3 = df.transform(lambda x : np.sqrt(x))
print(ex3)

          col1      col2      col3
row1  3.162278  6.324555  8.366600
row2  4.472136  7.071068  8.944272
row3  5.477226  7.745967  9.486833


In [91]:
#여러 함수를 동시에 적용하는 경우 list나 dict형태로 func값을 입력하는 경우 여러 함수를 동시에 적용할 수 있습니다.
#list로 입력하는 경우 마치 multi index처럼 multi columns가 생성됩니다.
ex4 = df.transform(['exp','sqrt'])
print(ex4)

              col1                    col2                    col3          
               exp      sqrt           exp      sqrt           exp      sqrt
row1  2.202647e+04  3.162278  2.353853e+17  6.324555  2.515439e+30  8.366600
row2  4.851652e+08  4.472136  5.184706e+21  7.071068  5.540622e+34  8.944272
row3  1.068647e+13  5.477226  1.142007e+26  7.745967  1.220403e+39  9.486833


In [92]:
ex5 = df.transform({'col2':'exp','col1':'sqrt'})
print(ex5)

              col2      col1
row1  2.353853e+17  3.162278
row2  5.184706e+21  4.472136
row3  1.142007e+26  5.477226


<h3>03-06. 문자열 형식의 계산식 적용 (eval)</h3>

DataFrame.eval(expr, inplace=False, kwargs)

eval메서드는 파이썬의 eval 메서드와 사용목적이 동일합니다. 문자열로 된 계산식을 적용합니다.

expr : 문자열 형태의 계산식입니다.</br>
inplace : {True / False} 계산된 값이 원본을 변경할지의 여부입니다. 기본적으로 원본은 변경되지 않습니다.</br>

In [96]:
data = [[1,2,3],[4,5,6],[7,8,9]]
col = ['col1','col2','col3']
row = ['row1','row2','row3']
df = pd.DataFrame(data = data, index = row, columns= col)
print(df)

      col1  col2  col3
row1     1     2     3
row2     4     5     6
row3     7     8     9


In [98]:
#기본적인 사용법 col1*col2-col3의 값을 갖는 col4를 만들어보겠습니다. 'col4'='col1'+'col2'-'col3'를 문자열 그대로 사용합니다.
#계산이 적용된 col4열이 생성된 것을 확인할 수 있습니다.
print(df.eval('col4=col1*col2-col3'))

      col1  col2  col3  col4
row1     1     2     3    -1
row2     4     5     6    14
row3     7     8     9    47


In [100]:
print(df)

      col1  col2  col3
row1     1     2     3
row2     4     5     6
row3     7     8     9


In [102]:
print(df.eval('col4=col1*col2-col3',inplace=True))
print(df)

None
      col1  col2  col3  col4
row1     1     2     3    -1
row2     4     5     6    14
row3     7     8     9    47
